In [ ]:
import csv
import os
import pandas as pd
import collections
import ast
import time
import collections
import json
import sqlite3
import xxhash
import numpy as np
import pickle 
import random
import glob
from io import StringIO
from pathlib import Path
from openai import OpenAI
import openai
print(openai.__version__)
os.environ["OPENAI_API_KEY"] = 

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [ ]:
from prompts.all_prompts import *
from order_by.sorting import *
from order_by.utils import *

In [ ]:
# sort ints
# what should the external memory size be?
random.seed(0)
random_list = random.sample(range(1, 2**16), 1024)
gold_list = sorted(random_list)

inversion = num_inversions(gold_list, random_list)
off = num_out_of_place(gold_list, random_list)
print(inversion, off)

In [ ]:
# gpt-4o should use about 128 as memoery size
# gpt-4o-mini should use about 8 as memory size
print("api calls should be 1 as the data size == chunk size")
print("api calls is at most 3 as it tries at most 3 times if LLM outputs have issue")
for modelname in ['gpt-4o-mini']: #'gpt-4o', 'gpt-4o-mini'
    bubble_sort_inversions_and_calls = []
    merge_sort_inversions_and_calls = []
    memory_sizes = [32,64,128,256]
    if modelname == 'gpt-4o-mini':
        memory_sizes = [8, 16, 32]
    for chunk_size in memory_sizes:
        print('chunck size: ', chunk_size)
        gold_list = sorted(random_list[:chunk_size])
        sorted_data, num_api_calls = external_bubble_sort(random_list[:chunk_size], external_comparisons, chunk_size, client, sort_int_prompt_template, modelname)
        print(sorted_data)
        inversions = num_inversions(gold_list, sorted_data)
        off = num_out_of_place(gold_list, sorted_data)
        bubble_sort_inversions_and_calls.append((inversions, num_api_calls, off))
        print(f"inversions: {inversions}, api calls: {num_api_calls} and out of place: {off}")
        
        sorted_data, num_api_calls = external_merge_sort(random_list[:chunk_size], external_comparisons, chunk_size, client, sort_int_prompt_template, modelname)
        print(sorted_data)
        inversions = num_inversions(gold_list, sorted_data)
        off = num_out_of_place(gold_list, sorted_data)
        merge_sort_inversions_and_calls.append((inversions, num_api_calls, off))
        print(f"inversions: {inversions}, api calls: {num_api_calls} and out of place: {off}")
        
    print("***************")
    print(modelname)
    print(bubble_sort_inversions_and_calls )
    print(merge_sort_inversions_and_calls)
    print("***************")

In [ ]:
def sortedness(gold_list, sorted_data):
    inversions = num_inversions(gold_list[:], sorted_data[:])
    off = num_out_of_place(gold_list[:], sorted_data[:])
    return inversions, off

In [ ]:
def store_metadata(d, sorted_data, inversions, outofplaces, api_calls, tokens, memory_size = 0):
    if memory_size == 0:
        d['sorted_data'] = sorted_data
        d['inversions'] = inversions
        d['out_of_places'] = outofplaces
        d['api_calls'] = api_calls
        d['tokens'] = tokens
    else:
        d[memory_size] = {}
        d[memory_size]['sorted_data'] = sorted_data
        d[memory_size]['inversions'] = inversions
        d[memory_size]['out_of_places'] = outofplaces
        d[memory_size]['api_calls'] = api_calls
        d[memory_size]['tokens'] = tokens
        
        
    
def experiment(dataname, column_to_sort, log_filename, pointwise_prompt_template,\
               external_pointwise_prompt_template, external_comparison_prompt_template,\
               pairwise_comparison_prompt_template):
    random.seed(0)
    
    fname = f"sorted_{dataname}.csv"
    file_path = f"./sorted_data/{fname}" 
    data = pd.read_csv(file_path)
    print(data.head())
    print(len(data))
    
    gold_list = data[column_to_sort].tolist()
    shuffled_list = random.sample(gold_list, len(gold_list)) # shuffle the sorted list
#     print(gold_list, shuffled_list)

    results = {}
    for modelname in ["gpt-4o-mini", "gpt-4o"]:
        results[modelname] = {'pointwise_sort':{}, 'quick_sort':{}, 'heap_sort':{},\
                              'external_pointwise_sort':{}, 'external_bubble_sort':{},\
                              'external_merge_sort':{}}
        out_t = float
        memory_sizes = [32, 64, 128]
        if modelname == 'gpt-4o-mini':
            memory_sizes = [4, 8]
        print(modelname)
        
        sorted_data, num_api_calls, tokens = pointwise_sort(shuffled_list, client, pointwise_prompt_template, modelname, out_t)
        inversions, off = sortedness(gold_list, sorted_data)
        print("pointwise_sort")
        print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
        store_metadata(results[modelname]['pointwise_sort'], sorted_data, inversions, off, num_api_calls, tokens)

        for m in memory_sizes:
            sorted_data, num_api_calls, tokens = external_pointwise_sort(shuffled_list, external_values, m, client, external_pointwise_prompt_template, modelname, out_t)
            inversions, off = sortedness(gold_list, sorted_data)
            print(f"external_pointwise_sort {m}")
            print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
            store_metadata(results[modelname]['external_pointwise_sort'], sorted_data, inversions, off, num_api_calls, tokens, m)


            sorted_data, num_api_calls, tokens = external_bubble_sort(shuffled_list, external_comparisons, m, client, external_comparison_prompt_template, modelname)
            inversions, off = sortedness(gold_list, sorted_data)
            print(f"external_bubble_sort {m}")
            print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
            store_metadata(results[modelname]['external_bubble_sort'], sorted_data, inversions, off, num_api_calls, tokens, m)


            sorted_data, num_api_calls, tokens = external_merge_sort(shuffled_list, external_comparisons, m, client, external_comparison_prompt_template, modelname)
            inversions = num_inversions(gold_list, sorted_data)
            off = num_out_of_place(gold_list, sorted_data)
            print(f"external_merge_sort {m}")
            print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
            store_metadata(results[modelname]['external_merge_sort'], sorted_data, inversions, off, num_api_calls, tokens, m)

        sorted_data, num_api_calls, tokens = quick_sort(shuffled_list, client, pairwise_comparison_prompt_template, modelname)
        inversions, off = sortedness(gold_list, sorted_data)
        print("quick_sort")
        print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
        store_metadata(results[modelname]['quick_sort'], sorted_data, inversions, off, num_api_calls, tokens)


        sorted_data, num_api_calls, tokens = heap_sort(shuffled_list, client, pairwise_comparison_prompt_template, modelname)
        inversions, off = sortedness(gold_list, sorted_data)
        print("heap_sort")
        print(f"inversions: {inversions}, api calls: {num_api_calls}, tokens: {tokens}, and out of place: {off}\n")
        store_metadata(results[modelname]['heap_sort'], sorted_data, inversions, off, num_api_calls, tokens)
    
    print(results)
    with open(log_filename, "w") as log_file:
        json.dump(results, log_file, indent=4) 


In [ ]:
dataname = "nba_heights"
column_to_sort = "full_name"
log_filename = "./logs/nba_heights.json"
experiment(dataname, column_to_sort, log_filename, nba_pointwise_prompt_template,\
               nba_external_pointwise_prompt_template, nba_external_comparison_prompt_template,\
               nba_pairwise_comparison_prompt_template)

In [ ]:
log_filename = "./logs/nba_heights.json"
with open(log_filename, "r") as log_file:
    loaded_data = json.load(log_file)
        
for modelname in loaded_data.keys():
    print()
    print(modelname)
    
    for external_function in ['external_pointwise_sort', 'external_bubble_sort', 'external_merge_sort']:
        print()
        for m in loaded_data[modelname][external_function].keys():
            print(external_function, f'memory size {m}')
            for field in ['inversions', 'out_of_places', 'api_calls']: #'sorted_data'
                print(field, loaded_data[modelname][external_function][m][field])
    
    for function_name in ['pointwise_sort', 'quick_sort','heap_sort']:
        print()
        print(function_name)
        for field in ['inversions', 'out_of_places', 'api_calls']: #'sorted_data'
            print(field, loaded_data[modelname][function_name][field])

In [ ]:
loaded_data[modelname]['external_pointwise_sort']